In [56]:
import pandas as pd
import gensim
import multiprocessing
import numpy as np
from utils import pickle_obj, semantic_search_author, semantic_search_word, get_related_authors, get_related_words, translate_dict
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from firebase_admin import credentials, firestore
import firebase_admin


In [57]:
df_kth = pd.read_csv("assets/dataframes/all_authors_df_2004")
df_su = pd.read_csv("assets/dataframes/suDf")
df_uppsala = pd.read_csv("assets/dataframes/uppsalaDf")
df_sodertorn = pd.read_csv("assets/dataframes/sodertornDf")

df_kth = df_kth.rename(columns={"KTH_id": "Auth_id", "KTH_name": "Auth_name"})
                    
#df_auth = pd.read_csv("assets/dataframes/KT_auth_2004")

In [58]:
def get_nlp_data(df):
    return df.Abstracts.values, df.Doc_id.values, df.Auth_id.values, df.Auth_name.values
    

In [59]:
text_doc_kth, doc_id_kth, auth_kth, name_kth = get_nlp_data(df_kth)
text_doc_su, doc_id_su, auth_su, name_su = get_nlp_data(df_su)
text_doc_uppsala, doc_id_uppsala, auth_uppsala, name_uppsala = get_nlp_data(df_uppsala)
text_doc_sodertorn, doc_id_sodertorn, auth_sodertorn, name_sodertorn = get_nlp_data(df_sodertorn)

In [60]:
TEXT = np.concatenate([text_doc_kth, text_doc_su, text_doc_uppsala, text_doc_sodertorn])
DOCID = np.concatenate([doc_id_kth, doc_id_su, doc_id_uppsala, doc_id_sodertorn]).astype(str)
AUTHID = np.concatenate([auth_kth, auth_su, auth_uppsala, auth_sodertorn ])
NAME = np.concatenate([name_kth, name_su, name_uppsala, name_sodertorn ])

In [61]:
df = pd.DataFrame(data=list(zip(TEXT, AUTHID, DOCID, NAME)), columns=["Abstracts", "Auth_id", "Doc_id", "Auth_name"])

In [104]:
df

,Abstracts,Auth_id,Doc_id,Auth_name
0,A model for the enhanced water dissociation th...,u16fm297:u13dx9f6,65923903,"Dahlkild, Anders A.:Behm, Marten"
1,Wood from white spruce Picea glauca that had ...,u16k1pmb:u1lkf75c,40520967,"Zhang, Liming:Henriksson, Gunnar"
2,"We consider the moduli space H-g,H-n of n-poin...",u1mv0zlg,41290309,"Bergstrom, Jonas"
3,In this paper a day-ahead planning algorithm f...,u16u3erw:u1naf2f4:u1fjok0u,40129874,"Matevosyan, Julija:Olsson, Magnus:Soder, Lennart"
4,"In this letter, we develop a fixed-point arith...",u1s42xk3:u12s8cr8,61385618,"Johansson, Christopher:Lansner, Anders B."
...,...,...,...,...
140172,Cette contribution décrit l'évolution du mouve...,usodertorn4324,9191_sodertorn,"Östberg, Kjell"
140173,The first part gives a chronological overview ...,usodertorn4324,9204_sodertorn,"Östberg, Kjell"
140174,Det första fallet av aids i Sverige diagnostis...,usodertorn4324,9213_sodertorn,"Östberg, Kjell"
140175,For decades “Swedology” was a rich and polemic...,usodertorn4356,9218_sodertorn,"Östlund, David"


In [102]:
!ls "assets/"

SU		 doc2vecModel
SemiSupArray	 finalproduct
assignmentStats  suNameId
dataframes	 terran-fc671-firebase-adminsdk-k8xbh-0d575a0a99.json
dictionaries	 uppsalaNameId
divaScrape	 words.json
diva_2004	 words.txt


In [103]:
df.to_csv("assets/df_all_schools")

In [87]:
auth_su

array(['uSU14622:uSU92268:uSU62150', 'uSU38475',
       'uSU38475:uSU54522:uSU54709:uSU83623', ...,
       'uSU9392:uSU15459:uSU86959', 'uSU79401', 'uSU79401'], dtype=object)

In [89]:
"uSU14622:uSU92268:uSU62150".replace("uSU", "usu").replace("uUppsala", "uuppsala")

'usu14622:usu92268:usu62150'

In [86]:
'uUppsala160669:uUppsala3191:uUppsala133542:uUppsala64943:uUppsala167930'.replace("uUppsala", "uuppsala")

'uuppsala160669:uuppsala3191:uuppsala133542:uuppsala64943:uuppsala167930'

In [92]:
reformat_str = [v.replace("uSU", "usu").replace("uUppsala", "uuppsala") for v in df.Auth_id]

In [93]:
doc_to_id = dict(zip(df.Doc_id, reformat_str))

In [95]:
doc_to_id

KeyError: 0

In [80]:
!pwd

/home/ekvall/kthLife/python


In [74]:
!ls "assets/dictionaries/"

auths_to_all_id_2004.pkl  id_to_auth_SU.pkl	    id_to_auth_uppsala.pkl
id_to_all_auths_2004.pkl  id_to_auth_sodertorn.pkl


In [96]:
import pickle

In [97]:
pickle.dump(doc_to_id, open("./assets/dictionaries/doc_to_id.pkl","wb"))

In [62]:
df.to_csv("assets/dataframes/KTH_UPPSALA_SODERTORN_SU")

In [8]:
def read_corpus(abstracts, doc, auth):
    for d, w, a in zip(abstracts, doc, auth):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(d), [str(w)] + a.split(":"))

In [9]:
train_corpus = list(read_corpus(TEXT, DOCID, AUTHID))

In [10]:
vocuabolary = list()
for corpus in train_corpus:
    vocuabolary +=  corpus.words

In [11]:
unique_words = list(set(vocuabolary))

In [12]:
dict_word =list()
for w in unique_words:
    dict_word.append({"word": w})

In [13]:
cred = credentials.Certificate("./assets/terran-fc671-firebase-adminsdk-k8xbh-0d575a0a99.json")
app = firebase_admin.initialize_app(cred)

store = firestore.client()


In [52]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec


class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
    
    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0

    def on_epoch_end(self, model):
        output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
        model.save(output_path)
        self.epoch += 1
        
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    
    def __init__(self):
        self.epoch = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
    
    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [53]:
epoch_logger = EpochLogger()

In [ ]:
cores = multiprocessing.cpu_count()

model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=1, dm=0,
                                      sample=1e-3, negative=15,hs=0,dbow_words=1,
                                      max_vocab_size=None,workers=cores,window=10,
                                          callbacks=[epoch_logger])

model.build_vocab(train_corpus)


I have pre-trained a model with vectordim=500, and with 10_000 iterations. The rest of the hyper-parameters are the same as in the implementation above.

In [ ]:
import time

start = time.time()
model.train(train_corpus, total_examples=model.corpus_count, epochs=1,report_delay=1)
end = time.time()
print(end - start)

In [ ]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = gensim.models.doc2vec.Doc2Vec.load(fname)  # you can continue training with the 

In [54]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("doc2vec_more_school_1000")
model = gensim.models.doc2vec.Doc2Vec.load(fname)  # you can continue training with the 

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [55]:
model.save("assets/doc2vecModel/more_school_1000")

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model = gensim.models.doc2vec.Doc2Vec.load(get_tmpfile("doc2vec_more_school"))  # you can continue training with the 

In [ ]:
import time

start = time.time()
model.train(train_corpus, total_examples=model.corpus_count, epochs=1,report_delay=1)
end = time.time()
print(end - start)

In [ ]:
324.3300178050995 * 1000 / 3600 / 24

In [ ]:
semantic_search_author(sentence, model, df, topn=300)

In [ ]:
#model.train(train_corpus, total_examples=model.corpus_count, epochs=1,report_delay=1)
#model = gensim.models.Word2Vec.load("assets/doc2vecModels/KTH2004_i10000_w10_d500_plainTrain/KTH2004_i10000_w10_d500_plainTrain.model")

Import dictonaries

In [ ]:
pickle_o = pickle_obj()
id_to_auth_kth = pickle_o.load("./assets/dictionaries/id_to_all_auths_2004")
id_to_auth_SU = pickle_o.load("./assets/dictionaries/id_to_auth_SU")
id_to_auth_uppsala = pickle_o.load("./assets/dictionaries/id_to_auth_uppsala")
id_to_auth_sodertorn = pickle_o.load("./assets/dictionaries/id_to_auth_sodertorn")



In [ ]:
id_to_auth = {}
for d in (id_to_auth, id_to_auth_SU, id_to_auth_uppsala, id_to_auth_sodertorn): 
    id_to_auth.update(d)
auth_to_id = {k:v for v, k in id_to_auth.items()}

In [ ]:
df

In [ ]:
def semantic_search_author(sentence, model, df, topn=30):                      
    word_2_vec = 0                                                             
    for word in sentence:                                                      
        word_2_vec += model[str(word)]                                         
    for a in model.docvecs.most_similar( [ word_2_vec ], topn=topn):           
        if a[0][0] !="u":                                                      
            print(str(a[0]),"||",get_article_authors_name(str(a[0]),df)," || ", np.around(a[1],2))

In [ ]:
def get_article_authors_name(doc_id, df):                                      
    name = ""                                                                  
    if len(df[df.Doc_id == doc_id].Auth_name) > 0:                              
        name_list = df[df.Doc_id == doc_id].Auth_name.values[0].split(":")      
        for i, n in enumerate(name_list):                                      
            if i == 0:                                                         
                name += str(n)                                                 
            elif i == len(name_list) - 1 and i > 1:                            
                name += " and " + str(n)                                       
            elif i > 1:                                                        
                name += " , " + str(n)                                         
    else:                                                                      
        name = "NaN"                                                           
                                                                               
    return name

# Check the success of the training

## Semantic serach after authors.

In [ ]:
sentence = ["dogs","forensic"]

In [ ]:
df[df.Doc_id == "38852_SU"].Abstracts.values

In [ ]:
semantic_search_author(sentence, model, df, topn=300)

In [ ]:
int(a[0])

In [ ]:
get_article_authors_name(str(a[0]),df)

In [ ]:
semantic_search_author(sentence, model, df, topn=30)

## Semantic serach after authors.

In [ ]:
semantic_search_word(sentence, model, df, topn=10)

## Search for related authors

In [ ]:
get_related_authors("Kall, Lukas(u1gqsept)",model, auth_to_id, id_to_auth, topn=20)

In [ ]:
get_related_authors("Savolainen, Peter",model, auth_to_id, id_to_auth, topn=20)

## Search for related words

In [ ]:
get_related_words("Kall, Lukas(u1gqsept)", model, auth_to_id, id_to_auth, topn=20)

In [ ]:
get_related_words("Savolainen, Peter", model, auth_to_id, id_to_auth, topn=20)

In [ ]:
get_related_words("Hudson, Paul", model, auth_to_id, id_to_auth, topn=20)

# Make a plot of the tSNE manifold to verify a underlying structure of the authorvectors

Get all authors-ids and its vectors

In [ ]:
cat = list()
docVec = list()
for k, v in zip(model.docvecs.doctags.keys(), model.docvecs.doctag_syn0): 
    if k[0] == "u":
        cat.append(id_to_auth[k])
        docVec.append(v)
docVec = np.asarray(docVec)

Find what department author is a member of.

In [ ]:
dep_names = list()
for n in cat: 
    if 0 < df_auth[df_auth.Name.str.contains(str(n))].dep_name.describe().unique()[0]:
        dep_names.append(df_auth[df_auth.Name.str.contains(str(n))].dep_name.describe().top)
    else:
        dep_names.append("nan")

Make sure that the deprtment names have is trandlated into ascii

In [ ]:
dep_names = np.asarray(dep_names)
name_dict = dict()
for u_id in np.unique(dep_names):
    n = str()
    if u_id is np.nan:
        u_id = str(u_id)
    for l in u_id:
        if ord(l) < 128:
            n +=l
        else:
            n += translate_dict[l]
    name_dict[str(u_id)] = str(n)
    
asci_cat = list()
for c in dep_names:asci_cat.append(name_dict[str(c)])

Make tSNE over author vectors

In [ ]:
tsne_input = docVec
tsne_input = pd.DataFrame(tsne_input, index=asci_cat)
tsne_input = tsne_input
tsne = TSNE(perplexity=5)
%time tsne_vectors = tsne.fit_transform(tsne_input)
tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])
tsne_vectors[u'word'] = tsne_vectors.index

Make the plot

## The plot seems to have an underlying structure, check it out!

In [ ]:
output_notebook()
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')


tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

In [ ]:
output_file("./assets/figures/tSNEauthors.html")
save(tsne_plot)

# Testing some SVD

In [ ]:
def get_U(V, comp=90):
    U, _, _ = svds(V, k=comp)
    norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
    U /= np.maximum(norms, 1e-7)
    return U

In [ ]:
U = get_U(model.wv.vectors)

In [ ]:
words = model.wv.index2word
x2i = {w:i for i, w in enumerate(words)}
i2x = {i:w for i, w in enumerate(words)}

In [ ]:
k = 5
for x in ['dogs', 'biofuels', 'peptide',"percolator","forensic","economic",
          "finance","algebra","convergence","bacteria"]:

    dd = cosine_similarity(U,U[x2i[x]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2x[i] == x: continue
        xy = tuple(sorted((x, i2x[i])))
        s += '(%s, %.3lf) ' % (i2x[i], dd[i])
    print('%s, %s' % (x, s))


In [ ]:
Ua = get_U(model.docvecs.doctag_syn0)

In [ ]:
authors = model.docvecs.index2entity
x2iA = {w:i for i, w in enumerate(authors)}
i2xA = {i:w for i, w in enumerate(authors)}

In [ ]:
k = 15
for x in ["Kall, Lukas(u1gqsept)", "Savolainen, Peter", "Hudson, Paul"]:

    dd = cosine_similarity(Ua,Ua[x2iA[auth_to_id[str(x)]]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2xA[i] == auth_to_id[str(x)]: 
            continue

        if i2xA[i][0] == "u":
            s += '(%s, %.3lf) ' % (id_to_auth[i2xA[i]], dd[i])
    print('%s, %s' % (x, s))

In [ ]:
Ub = get_U(np.concatenate((model.docvecs.doctag_syn0, model.wv.vectors)))

In [ ]:
both = authors + words
x2iB = {w:i for i, w in enumerate(both)}
i2xB = {i:w for i, w in enumerate(both)}

In [ ]:
k = 25
for x in ["Kall, Lukas(u1gqsept)", "Savolainen, Peter", "Hudson, Paul"]:

    dd = cosine_similarity(Ub,Ub[x2iB[auth_to_id[str(x)]]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2xB[i] == x: continue
        try:
            int(i2xB[i][0])
        except:
            if not i2xB[i][0] == "u" and not i2xB[i][1] == "l":
                s += '(%s, %.3lf) ' % (i2xB[i], dd[i])
    print('%s, %s' % (x, s))